In [11]:
from kafka import KafkaConsumer
from pymongo import MongoClient,errors
import pprint
import json
import time
import os
import pandas as pd

def mongodb_connect(domain, port):
    domain_str = str(domain) + ":" + str(port)
    try:
        print ("Trying to connect to MongoDB server:", domain, "on port:", port)
        client = MongoClient(host = [domain_str],
                             serverSelectionTimeoutMS = 2000)
        print ("server_info():", client.server_info())
    except errors.ServerSelectionTimeoutError as err:
        print ("pymongo ERROR:", err)
        client = None
    return client

def consumer_multi_collection(name_db,topic_list):
    consumer = KafkaConsumer(*topics, bootstrap_servers='localhost:9092', 
                             auto_offset_reset='earliest')#,group_id="historian-monitor"
    for message in consumer:
        msg = message.value.decode()
        tagname = msg.split(";")[1]
        mycollection=name_db[tagname]
        print("------------")
        mongodata={
            'count':msg.split(":")[0],
            'timestamp':msg.split(";")[0].split(":",1)[1],
            'tagname':tagname,
            'value':msg.split(";")[2],
            'quality':msg.split(";")[3]
        }
        print(mongodata)
        #result=mycollection.insert_one(mongodata)
    #consumer.close()
if __name__=="__main__":
    client = mongodb_connect("localhost",28018)
    topics=["CRY.TGBT_NORMAL.CRY_rea_cons_pow",
            "CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow"]
    db=client.test
    consumer_multi_collection(db,topics)

Trying to connect to MongoDB server: localhost on port: 28018
server_info(): {'version': '4.2.3', 'gitVersion': '6874650b362138df74be53d366bbefc321ea32d4', 'modules': [], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [4, 2, 3, 0], 'openssl': {'running': 'OpenSSL 1.0.1e-fips 11 Feb 2013', 'compiled': 'OpenSSL 1.0.1e-fips 11 Feb 2013'}, 'buildEnvironment': {'distmod': 'rhel70', 'distarch': 'x86_64', 'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.2.0', 'ccflags': '-fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -Werror -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -fno-builtin-memcmp', 'cxx': '/opt/mongodbtoolchain/v3/bin/g++: g++ (GCC) 8.2.0', 'cxxflags': '-Woverloaded-virtual -Wno-maybe-uninitialized -fsized-deallocation -std=c++17'

KeyboardInterrupt: 